In [28]:
# dataset check 
# del  empty .json     no-label-data 
import os, json 

def check_dataset(data_root, del_emptyJson=True, del_unmatchData=True):
    jpg_nameset = {f.split('.')[0] for f in os.listdir(data_root) if f.endswith('.jpg')}
    json_nameset = {f.split('.')[0] for f in os.listdir(data_root) if f.endswith('.json')}

    jpg_no_json = {f for f in jpg_nameset if f not in json_nameset}
    json_no_jpg = {f for f in json_nameset if f not in jpg_nameset}
    
    print(f"{jpg_no_json=}")
    print(f"{json_no_jpg=}")
    
    if del_unmatchData:
        for na in jpg_no_json:
            jpg_path = os.path.join(data_root, na+'.jpg')
            # os.remove(jpg_path)
            print(f"unmatched {jpg_path=}")
        for na in json_no_jpg:
            json_path = os.path.join(data_root, na+'.json')
            # os.remove(json_path)
            print(f"unmatched{json_path=}")
        
    # if json-file no label information   delete json-file
    if del_emptyJson:
        for json_name in json_nameset:
            json_path = os.path.join(data_root, json_name+".json")
            with open(json_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                if 'shapes' not in data or not data['shapes']:
                    # os.remove(json_path)
                    print(f"{json_path=}")
        
dr = r"F:\jueyuanzi\zyanshou\code_projection\jueyuanzi\VOCdevkit\datasets\wh1600"
check_dataset(dr)

jpg_no_json=set()
json_no_jpg=set()


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\jueyuanzi\\zyanshou\\code_projection\\jueyuanzi\\VOCdevkit\\datasets\\wh1600\\image(1315)_R1C0.json'

In [5]:
# image info  --jpg   3000x2250: 734   3000x1688: 17
from PIL import Image
from collections import defaultdict
import tqdm  

def jpgs_info(jpgs_dir, writeTxt=False):
    size_count = defaultdict(int)  # 用于统计各个尺寸的图片数量
    for im in tqdm.tqdm(os.listdir(jpgs_dir)):
        if im.endswith('.jpg'):
            img_path = os.path.join(jpgs_dir, im)
            image = Image.open(img_path)
            size = image.size
            size_count[size] += 1
        else:
            continue
            # print("not jpg img:", im)
    if writeTxt:
        with open(f"{os.path.dirname(jpgs_dir)}/jpgsinfo.txt", 'w') as file:
            file.write("图片尺寸和数量： \n")
            for size, count in size_count.items():
                file.write(f"尺寸 {size[0]}x{size[1]}: {count} 张\n")
    else:
        print("图片尺寸和数量： \n")
        for size, count in size_count.items():
            print(f"size {size[0]}x{size[1]}: {count}")

jd = r"F:\jueyuanzi\zyanshou\dataset\kk"
jpgs_info(jd)

  0%|          | 0/1502 [00:00<?, ?it/s]

not jpg img: 1-20241219101309948082415392237490-B_0.json
not jpg img: 1-20241220113701392775847463121656-B_1.json
not jpg img: 10-20241217091709336632986136527647-B_0.json
not jpg img: 10-20241220105512444955792731152719-B_1.json
not jpg img: 100-20241215121547714857647117654938-B_1.json
not jpg img: 101-20241215121453028843499640992518-B_1.json
not jpg img: 102-20241215121423806258805093028326-B_1.json
not jpg img: 103-20241215121423806258805093028326-B_1.json
not jpg img: 104-20241215121343606167641593861688-B_1.json
not jpg img: 105-20241215110806273814540369156308-B_0.json
not jpg img: 106-20241215110754021983137157957043-B_0.json
not jpg img: 107-20241215110754021983137157957043-B_0.json
not jpg img: 108-20241215110739360704741071974596-B_0.json
not jpg img: 109-20241215110739360704741071974596-B_0.json
not jpg img: 11-20241216162641145724404533734601-B_0.json
not jpg img: 11-20241220105507273378645300645982-B_1.json
not jpg img: 110-20241215110728699217103849722095-B_0.json
not j

100%|██████████| 1502/1502 [00:00<00:00, 3777.66it/s]

not jpg img: image(1301).json
not jpg img: image(1303).json
not jpg img: image(1304).json
not jpg img: image(1305).json
not jpg img: image(1306).json
not jpg img: image(1307).json
not jpg img: image(1308).json
not jpg img: image(1309).json
not jpg img: image(1311).json
not jpg img: image(1312).json
not jpg img: image(1313).json
not jpg img: image(1314).json
not jpg img: image(1315).json
not jpg img: image(1316).json
not jpg img: image(1317).json
not jpg img: image(1318).json
not jpg img: image(1319).json
not jpg img: image(1321).json
not jpg img: image(1326).json
not jpg img: image(1327).json
not jpg img: image(1328).json
not jpg img: image(1330).json
not jpg img: image(1331).json
not jpg img: image(1332).json
not jpg img: image(1333).json
not jpg img: image(1334).json
not jpg img: image(1335).json
not jpg img: image(1336).json
not jpg img: image(1337).json
not jpg img: image(1338).json
not jpg img: image(1348).json
not jpg img: image(1350).json
not jpg img: image(1352).json
not jpg im

In [25]:
# labelme .json  ->  mask  
from PIL import ImageDraw
import base64

# 定义类别和对应的颜色
classes = ["_background_", "jyz", "baodian"]
colors = {
    "_background_": (0, 0, 0),
    "jyz": (255, 0, 0),      # 红色
    "baodian": (0, 255, 0)   # 绿色
}

def json2mask(json_path, mask_path, classes, colors):
    with open(json_path, 'r', encoding='utf-8')as f:
        data = json.load(f)
    
    img_width = data.get('imageWidth', 0)
    img_height = data.get('imageHeight', 0)
    if img_width == 0 or img_height ==0:
        raise ValueError(f"Invalid image size in {json_path}")

    mask = Image.new('P', (img_width, img_height), color=0)
    
    palette = []
    for cls in classes:
        palette.extend(colors[cls])
    palette += [0,0,0] * (256-len(classes))
    mask.putpalette(palette)
    
    draw = ImageDraw.Draw(mask)
    
    for shape in data.get('shapes', []):
        label = shape.get('label')
        if label not in classes:
            print(f"Unknown label '{label}' in {json_path}, skipping...")
            continue
        points = shape.get('points', [])
        if not points:
            continue
        polygon = [tuple(map(int, point)) for point in points]
        class_index = classes.index(label)
        draw.polygon(polygon, fill=class_index)
    
    mask.save(mask_path, format='PNG')
  
def convert_dataset(data_root, output_dir)  :
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    for filename in tqdm.tqdm(os.listdir(data_root)):
        if filename.endswith('.json'):
            json_path = os.path.join(data_root, filename)
            mask_filename = filename.replace('.json', '.png')
            mask_path = os.path.join(output_dir, mask_filename)
            json2mask(json_path, mask_path, classes, colors)
       
# get jpg from json 
def get_jpg_json(jsonpath, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    json_filename = os.path.basename(jsonpath)
    jpg_filename = os.path.splitext(json_filename)[0] +'.jpg'
    jpg_path = os.path.join(output_dir, jpg_filename)
    
    with open(jsonpath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    image_data = data.get('imageData')
    image_bytes = base64.b64decode(image_data)
    with open(jpg_path, 'wb') as img_f:
        img_f.write(image_bytes)
    
json_d = r"F:\jueyuanzi\zyanshou\dataset\kk"
out_d = r"F:\jueyuanzi\zyanshou\dataset\kk" 
convert_dataset(json_d, out_d)


100%|██████████| 2253/2253 [01:21<00:00, 27.53it/s]


In [18]:
# check mask 
def get_color_palette(classes, colors):
    palette = []
    for cls in classes:
        palette.extend(colors[cls])
    palette += [0,0,] * (256-len(classes))
    return palette 

mask_path = r"F:\jueyuanzi\zyanshou\dataset\kk\1-20241219101309948082415392237490-B_0.png"
# Image.open(mask_path).convert("P").show()
# Image.open(mask_path).show()

mask = Image.open(mask_path)
print(f"{mask.mode=}")
palette = mask.getpalette()
# print(f"{palette=}")
mask_rgb = mask.convert("RGB")
print(f"{mask_rgb.mode=}")
mask_rgb.show()



mask.mode='P'
mask_rgb.mode='RGB'


In [23]:
def get_CropBboxDict(pilImage, splitSize, splitRow, splitCol):
    w, h = pilImage.size 
    sw, sh = splitSize
    
    # if image_h is small ..
    if h < 1700:
        sh = h  
        stride_w = (w-sw)/(splitCol-1)
        stride_h = 0
        splitRow = 1
    else:
        # stride * (n-1) + sw = pi
        stride_w = (w-sw)/(splitCol-1)
        stride_h = (h-sh)/(splitRow-1)
    
    def get_splitBbox(sw, sh, row_, col_, stride_w, stride_h):
        bboxes = {}
        for i in range(row_):
            x1 = i * stride_w
            x2 = x1 + sw 
            for j in range(col_):
                y1 = j * stride_h 
                y2 = y1 + sh 
                bboxes[(i, j)] = [int(x1), int(y1), int(x2), int(y2)]
        return bboxes
    
    bbox_dict = get_splitBbox(sw, sh, splitRow, splitCol, stride_w, stride_h)
    
    return bbox_dict

image_ = Image.open(r"F:\jueyuanzi\zyanshou\code_projection\jueyuanzi\VOCdevkit\datasets\images/image(676).jpg").convert('RGB')
dict_ = get_CropBboxDict(image_, (1600,1600), 2, 2)
print(dict_)
# 3000,1688 : {(0, 0): [0, 0, 1600, 1688], (0, 1): [0, 0, 1600, 1688]}
# 3000:2250 : {(0, 0): [0, 0, 1600, 1600], (0, 1): [0, 650, 1600, 2250], 
#              (1, 0): [1400, 0, 3000, 1600], (1, 1): [1400, 650, 3000, 2250]}




{(0, 0): [0, 0, 1600, 1600], (0, 1): [0, 650, 1600, 2250], (1, 0): [1400, 0, 3000, 1600], (1, 1): [1400, 650, 3000, 2250]}


In [26]:
def split_masks(data_root, save_dir):
    
    for im in tqdm.tqdm(os.listdir(data_root)):
        if im.endswith('.jpg'):
            name_ = im.split('.')[0]
            jpg_path = os.path.join(data_root, name_+'.jpg')
            png_path = os.path.join(data_root, name_+'.png')

            jpg_ = Image.open(jpg_path).convert('RGB')
            png_ = Image.open(png_path)
            bbox_dict = get_CropBboxDict(jpg_, (1600,1600), 2, 2)
            
            for k, box in bbox_dict.items():
                _suffix  = f"_R{k[0]}C{k[1]}"
                jpg_path_save = os.path.join(save_dir, name_+_suffix+'.jpg')
                jpg_.crop(box).save(jpg_path_save)
                png_path_save = os.path.join(save_dir, name_+_suffix+'.png')
                png_.crop(box).save(png_path_save)

da_r = r"F:\jueyuanzi\zyanshou\code_projection\jueyuanzi\VOCdevkit\datasets\images"
save_r = r"F:\jueyuanzi\zyanshou\code_projection\jueyuanzi\VOCdevkit\datasets\wh1600"
split_masks(da_r, save_r)             

100%|██████████| 1502/1502 [02:22<00:00, 10.55it/s]
